In [1]:
import keras
from bs4 import BeautifulSoup
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
import numpy as np
from keras.utils import to_categorical
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
from keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
DIRECTORY = r"dataset/"
CATEGORIES = ["with_mask", "without_mask"]
training_data = []
labels = []
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    indexes=CATEGORIES.index(category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image=preprocess_input(image)
        training_data.append(image)
        labels.append(indexes)

c:\users\shiva\appdata\local\programs\python\python38\lib\site-packages\PIL\Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [3]:
labels=to_categorical(labels)

In [4]:
training_data=np.array(training_data)
labels=np.array(labels)

In [5]:
(trainX, testX, trainY, testY) = train_test_split(training_data, labels,test_size=0.20, stratify=labels, random_state=42)

In [6]:
data_aug = ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")

In [7]:
from keras.models import Sequential,Model
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,BatchNormalization,Flatten,Input,AveragePooling2D


In [15]:
baseModel=MobileNetV2(include_top=False,input_tensor=Input(shape=(224,224,3)))

In [16]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

mask_detect = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False

In [17]:

mask_detect.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [18]:
(trainX, testX, trainY, testY) = train_test_split(training_data, labels,test_size=0.20, random_state=42)

In [19]:
from keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
checkpoint=ModelCheckpoint('model/',monitor='val_loss',mode='min',save_best_only=True,verbose=1)
early=EarlyStopping(monitor='val_loss',min_delta=0,patience=15,verbose=1,restore_best_weights=True)
callbacks=[checkpoint,early]
mask_detect.fit(data_aug.flow(trainX,trainY,batch_size=32),batch_size=32,epochs=10,validation_data=(testX,testY),callbacks=callbacks)

Epoch 1/10
87/87 [==============================] - ETA: 0s - loss: 0.1318 - accuracy: 0.9493
Epoch 00001: val_loss improved from inf to 0.03693, saving model to model\
INFO:tensorflow:Assets written to: model\assets
87/87 [==============================] - 47s 536ms/step - loss: 0.1318 - accuracy: 0.9493 - val_loss: 0.0369 - val_accuracy: 0.9797
Epoch 2/10
87/87 [==============================] - ETA: 0s - loss: 0.0558 - accuracy: 0.9822
Epoch 00002: val_loss improved from 0.03693 to 0.03163, saving model to model\
INFO:tensorflow:Assets written to: model\assets
87/87 [==============================] - 46s 531ms/step - loss: 0.0558 - accuracy: 0.9822 - val_loss: 0.0316 - val_accuracy: 0.9899
Epoch 3/10
87/87 [==============================] - ETA: 0s - loss: 0.0440 - accuracy: 0.9837
Epoch 00003: val_loss did not improve from 0.03163
87/87 [==============================] - 22s 251ms/step - loss: 0.0440 - accuracy: 0.9837 - val_loss: 0.0324 - val_accuracy: 0.9841
Epoch 4/10
87/87 [===

In [6]:
import cv2

cap=cv2.VideoCapture(0)
face_detect=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
while True:
    ret,frame=cap.read()
    detect_face=face_detect.detectMultiScale(frame,1.1,4)
    image = frame
    for (x,y,w,h) in detect_face:
        faceROI=image[y:y+h+50,x:x+50+w]
        arr=faceROI
        arr=cv2.resize(arr,(224,224))
        arr=arr/255.0
        arr=arr.reshape((1,224,224,3))
        pred=(np.argmax(mask_detect.predict(arr)))
#         print(pred)
        if pred==0:
            text='with_mask'
            cv2.putText(frame,text=text,org=(30,30),fontFace=cv2.FONT_HERSHEY_PLAIN,fontScale=1,thickness=2,color=(0,255,0))
        else:
            text='without_mask'
            cv2.putText(frame,text=text,org=(30,30),fontFace=cv2.FONT_HERSHEY_PLAIN,fontScale=1,thickness=2,color=(0,0,255))
        cv2.rectangle(frame,(x,y),(x+w,y+h),color=(255,0,0),thickness=1)
        
        cv2.imshow('mask',faceROI)
    cv2.imshow('farme',frame)
    if cv2.waitKey(1) & 0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()

In [67]:
print(mask_detect.evaluate(testX,testY))

22/22 [==============================] - 9s 403ms/step - loss: 0.0206 - accuracy: 0.9942
[0.0206452589482069, 0.9942113161087036]


In [2]:
from keras.models import load_model
import numpy as np

In [3]:
mask_detect=load_model('new_mask_model.h5')

In [4]:
print(mask_detect.evaluate(testX,testY))

NameError: name 'testX' is not defined